In [1]:
import pandas as pd
from src import bayesian_3pt_percentage_with_credible_interval
from sqlalchemy import create_engine

In [2]:
engine = create_engine("sqlite:///Data/player_database.db")
input_data = pd.read_sql("BAYESIAN_3PT_INPUT_2023_24", engine)
input_data = input_data.dropna()

In [18]:
players = []
bayesian_three_perc = []
lower_bounds = []
upper_bounds = []

In [19]:
for index, row in input_data.iterrows():
    if row["SEASON_3PA"] > 0:
        players.append(str(int(row["PLAYER_ID"])))
        perc, lower_bound, upper_bound = bayesian_3pt_percentage_with_credible_interval(float(row["HISTORICAL_3P%"]), row["SEASON_3PA"], row["SEASON_3PM"])
        bayesian_three_perc.append(perc)
        lower_bounds.append(lower_bound)
        upper_bounds.append(upper_bound)

In [20]:
out_dict = {"PLAYER": players, "3P%": bayesian_three_perc, "LOWER_BOUND": lower_bounds, "UPPER_BOUND": upper_bounds}
df = pd.DataFrame(out_dict)
df.to_sql("BAYESIAN_3PT_OUTPUT_2023_24", con=engine, if_exists="replace",index=False)

470

In [21]:
print(df)

      PLAYER       3P%  LOWER_BOUND  UPPER_BOUND
0    1630173  0.320755     0.223223     0.418418
1    1628389  0.129731     0.035035     0.227227
2    1630534  0.355263     0.257257     0.453453
3    1630583  0.318471     0.220220     0.416416
4    1629638  0.337296     0.239239     0.435435
..       ...       ...          ...          ...
465  1626174  0.378037     0.280280     0.476476
466  1626153  0.348142     0.250250     0.446446
467  1629027  0.351212     0.253253     0.449449
468  1630209  0.222223     0.124124     0.320320
469   203469  0.217689     0.120120     0.315315

[470 rows x 4 columns]


In [22]:
print(df[df["PLAYER"] == "2544"])

    PLAYER       3P%  LOWER_BOUND  UPPER_BOUND
201   2544  0.344507     0.246246     0.442442
